In [0]:
# Helpers
from Helpers import *
  
# For UploadSequence
from pyspark.sql.functions import unix_timestamp, monotonically_increasing_id, row_number
from pyspark.sql.window import Window

# For UploadDate
from pyspark.sql.functions import current_date, date_format

sc = SparkContext.getOrCreate()

## Input Parameters

In [0]:
BATCH = get_job_parameter('batch')

print(f"BATCH: {BATCH}")
print(f"AWS_REGION: {AWS_REGION}")
print(f"LOAD_DATE: {LOAD_DATE}")
print(f"STATUS_TABLE: {STATUS_TABLE}")
print(f"RWE_BUCKET: {RWE_BUCKET}")
print(f"SNS_TOPIC: {SNS_TOPIC}")
print(f"DMZ_CATALOG: {DMZ_CATALOG}")

READ_PATH = f's3://{RWE_BUCKET}/{LANDING_PATH}'

### Write

#### Write Processed Measurement

In [0]:
# Before writing, split the uploaddate into year/month/day and partition by those in that order
def write_measurement(df, file_type, libre_type):
    file_type = file_type.replace('-', '_') # Replace table-name to table_name for valid SQL table name convention
    if libre_type == "FSL3":  
        schema = FSL3_SCHEMA
    else:
        schema = FSL1_FSL2_SCHEMA

    try:
        df.write.mode('append') \
            .option('nullValue', None) \
            .option("mergeSchema", "true") \
            .saveAsTable(f"{DMZ_CATALOG}.{schema}.{file_type}")
    except Exception as e:
        print(e)
        SNSNotifier(AWS_REGION, SNS_TOPIC, LOAD_DATE).notify_error(str(e))
        sys.exit(f"Error writing file type: {file_type}")

#### Write Unprocessed Measurement

In [0]:
def write_unprocessed_records(df, file_type, mode):
    try:
        df.write.mode(mode) \
            .option('nullValue', None) \
            .parquet(f'{UNPROCESSED_PATH}/{file_type}/{BATCH}/')
    except Exception as e:
        print(e)
        SNSNotifier(AWS_REGION, SNS_TOPIC, LOAD_DATE).notify_error(str(e))
        sys.exit(f"Error writing unprocessed records")

# ETL

In [0]:
print("Start ETL")

### devices file

#### Raw device file

In [0]:
device_df = spark.read.parquet(f"{READ_PATH}/device/{BATCH}/")
device_df.cache()

#### Filter on DeviceTypeID

In [0]:
# Extract records for LibreLink and LDD
# DeviceType (40066, 40067): LibreLink
# DeviceType (40026, 40027): LDD
# DeviceType (90001, 90002): ConnectedDevice
# DeviceType (40068) : FSL3App
# DeviceType (40031) : FSL3Reader

device_with_type_df = device_df \
    .withColumn("deviceuuid", 
         when(col("DeviceTypeID").isin([40066, 40067, 40068]), col("SerialNumber")) \
        .when(col("DeviceTypeID").isin([40026, 40027]), concat(col("SerialNumber"), col("PatientID"))) \
        .when(col("DeviceTypeID").isin([40031]), concat(col("SerialNumber"), col("PatientID"))) \
        .when(col("DeviceTypeID").isin([90001, 90002]), col("SerialNumber")) \
    ) \
    .withColumn("readertype", 
        when(col("DeviceTypeID").isin([40066, 40067, 40068]), "MobileApp") \
        .when(col("DeviceTypeID").isin([40026, 40027, 40031]), "Reader")   \
    ) \
    .withColumn("devicetype",
        when(col("DeviceTypeID").isin([40066, 40067]), "LibreLink") \
        .when(col("DeviceTypeID").isin([40068]), "Libre3") \
        .when(col("DeviceTypeID").isin([40031]), "Libre3Reader") \
        .when(col("DeviceTypeID").isin([40026, 40027]), "LDD")      \
        .when(col("DeviceTypeID").isin([90001, 90002]), "ConnectedPen")      \
    ) \
    .withColumn("libretype", 
        when(col("devicetype").isin(["LibreLink", "LDD"]), "FSL1&2") \
        .when(col("devicetype").isin(["Libre3", "Libre3Reader"]), "FSL3") \
    ) \
    .withColumnRenamed("FirmwareVersion", "firmwareversion") \
    .withColumnRenamed("SystemTypeNum", "systemtype") \
    .withColumnRenamed("ModelName", "modelname") \
    .withColumnRenamed("ID", "DeviceID") \
    .withColumnRenamed("PatientID", "accountid")

# Capture all records not from Librelink or LDD systems, or not a ConnectedPen
unprocessed_devices_source_df = device_with_type_df.filter((col("devicetype").isNull()))

#### App Devices

In [0]:
# Process only app uploads with valid firmware version
        # FSLL (iPhone):  2.2.1 or 2.0.3 or 2.2.2 or 2.4 or 2.4.1 or 2.5 or 2.6 or 2.7 or 2.8 or 2.10 or 2.11 or 3.x
        # FSLL (Android): 2.3 or 2.0.2 or 2.2.2 or 2.4 or 2.4.1 or 2.5 or 2.6 or 2.7 or 21 or 2.8 or 2.10 or 2.11 or 3.x
app_df = device_with_type_df.filter(col("readertype") == "MobileApp")
        
filtered_app_df = app_df.filter(
    col("firmwareversion").like("2.2.1%") | col("firmwareversion").like("2.0.3%") | col("firmwareversion").like("2.3%")  | \
    col("firmwareversion").like("2.0.2%") | col("firmwareversion").like("2.2.2%") | col("firmwareversion").like("2.4%")  | \
    col("firmwareversion").like("2.5%")   | col("firmwareversion").like("2.6%")   | col("firmwareversion").like("2.7%")  | \
    col("firmwareversion").like("21%")    | col("firmwareversion").like("2.8%")   | col("firmwareversion").like("2.10%") | \
    col("firmwareversion").like("2.11%")  | col("firmwareversion").like("3.%")
)

# Get unprocessed app data
unprocessed_app_df = app_df.exceptAll(filtered_app_df)

# Apply logic to derive App systemtype
final_app_df = filtered_app_df.withColumn(
   "systemtype", when(col("modelname").startswith("com.freestylelibre"), "Android") \
    .when(col("modelname").startswith("com.abbott.libre"), "iOS") \
    .when(col("modelname").startswith("iPhone"), "iOS")
)

#### Reader Devices

In [0]:
# Process only reader uploads from supported readers - System A (FSL) and System E (FSL2)
        # System A: "JCG", "JGG", "JPG", "JKG", "JNG", "JCM", "JGM", "JPM", "JDG", "JTG", "JTM"
        # System E: "MAG", "MBG", "MCG", "MAM", "MBM", "MCM"

reader_df = device_with_type_df.filter(col("readertype") == "Reader")

filtered_reader_df = reader_df.filter(
    # System A Filters
    col("deviceuuid").startswith("JCG") | col("deviceuuid").startswith("JGG") | col("deviceuuid").startswith("JPG") | \
    col("deviceuuid").startswith("JKG") | col("deviceuuid").startswith("JNG") | col("deviceuuid").startswith("JCM") | \
    col("deviceuuid").startswith("JGM") | col("deviceuuid").startswith("JPM") | col("deviceuuid").startswith("JDG") | \
    col("deviceuuid").startswith("JTG") | col("deviceuuid").startswith("JTM") | \
    
    # System E Filters
    col("deviceuuid").startswith("MAG") | col("deviceuuid").startswith("MBG") | col("deviceuuid").startswith("MCG") | \
    col("deviceuuid").startswith("MAM") | col("deviceuuid").startswith("MBM") | col("deviceuuid").startswith("MCM")
)

# Get unprocessed reader data
unprocessed_reader_df = reader_df.exceptAll(filtered_reader_df)

# Mappings of systemTypeNum to systemType
# mapping = {"0": "A", "1": "B", "2": "C", "3": "D", "4": "E"} 

final_reader_df = filtered_reader_df.withColumn("systemtype", when(col("systemtype") == 0, 'A') \
        .when(col("systemtype") == 1, 'B') \
        .when(col("systemtype") == 2, 'C') \
        .when(col("systemtype") == 3, 'D') \
        .when(col("systemtype") == 4, 'E') \
        .otherwise(None)
    )

#### Connected Devices

In [0]:
connected_devices_df = device_with_type_df.filter(col("devicetype") == "ConnectedPen")

#### Stitch Device File Back Together & Pull Needed Cols

In [0]:
# header = ['accountid', 'deviceuuid', 'readertype', 'devicetype', 'firmwareversion', 'systemtype', 'DeviceID']

# LDD/LL Devices (for device-settings/account-devices
final_ldd_ll_devices = final_reader_df.union(final_app_df)

# df containing LDD/LL/ConnectedPen devices
# Used when creating insulin-device dataframe
all_final_devices_df = final_reader_df.union(final_app_df).union(connected_devices_df)
# all_final_devices_df.show()

# Deidentify LDD/LL Devices
hashed_ldd_ll_devices_df = final_ldd_ll_devices.withColumn('accountid', deid('accountid')).withColumn('deviceuuid', deid('deviceuuid'))

# Add uploaddate field
# hashed_ldd_ll_devices_df = hashed_ldd_ll_devices_df.withColumn("uploaddate", date_format(current_date(), date_string_format))

#### Get all unprocessed devices

In [0]:
# Get all unprocessed device records (includes ConnectedDevices)

all_unprocessed_devices_df = unprocessed_devices_source_df.union(unprocessed_app_df.union(unprocessed_reader_df))

total_count = device_df.count()
processed_count = all_final_devices_df.count()
unprocessed_count = all_unprocessed_devices_df.count()
print("All Source Devices Count: ", total_count)
print("Processed Devices Count: ", processed_count)
print("Unprocessed Devices Count: ", unprocessed_count)
print("Unidentified Devices Count: ", total_count - processed_count - unprocessed_count)

In [0]:
# Write unprocessed devices (rename fields back to source names)
print("Writing unprocessed device records")

all_unprocessed_devices_final_df = all_unprocessed_devices_df.withColumnRenamed("DeviceID", "ID") \
    .withColumnRenamed("systemtype", "SystemTypeNum") \
    .withColumnRenamed("accountid", "PatientID") \
    .select(*device_header)

write_unprocessed_records(all_unprocessed_devices_final_df, 'device', 'overwrite')

### measurement file

In [0]:
# Load the measurement file (lazy)
measurement_df = spark.read.parquet(f"{READ_PATH}/measurement/{BATCH}/")
measurement_df.cache()

### user file

In [0]:
# Load the user file (lazy)
user_df = spark.read.parquet(f"{READ_PATH}/user/{BATCH}/")
user_df.cache()

## Create account-devices.csv

#### Validate Attributes

In [0]:
valid_account_devices_df = hashed_ldd_ll_devices_df.filter(
    col("accountid").isNotNull() & (col("accountid") != "") & 
    col("deviceuuid").isNotNull() & (col("deviceuuid") != "")
)
valid_account_devices_df = valid_account_devices_df.withColumn("uploaddate", date_format(current_date(), date_string_format))

print(f"Valid account-devices count: {valid_account_devices_df.count()}")

#### Split FSL1&2 and FSL3 + Write

In [0]:
"""
Superior headers = ["accountid", "deviceuuid", "uploaddate"]
"""
# ad_headers = ["accountid", "deviceuuid", "uploaddate"]
ad_headers = ["accountid", "deviceuuid", "uploaddate"]

# Split FSL1&2 / FSL3
fsl1_account_devices_df = valid_account_devices_df.filter(col("libretype") == "FSL1&2")
fsl3_account_devices_df = valid_account_devices_df.filter(col("libretype") == "FSL3")

# Select final columns
account_devices_fsl1_df = fsl1_account_devices_df.select(*ad_headers)
account_devices_fsl3_df = fsl3_account_devices_df.select(*ad_headers)

# Write account-devices
write_measurement(account_devices_fsl1_df, 'account-devices', 'FSL1&2')
write_measurement(account_devices_fsl3_df, 'account-devices', 'FSL3')
print('Done writing Account-Devices')

## Create device-settings file

In [0]:
"""
headers = ["accountid", "deviceuuid", "devicenationality", "uploadsequence", "firmwareversion", "systemtype", "readertype", "devicetype", "uploaddate"]

From device file: deviceuuid, readertype, devicetype, firmwareversion, systemtype
From user file: devicenationality
KEY: accountid

To Derive: uploadsequence, uploaddate

FSL3: Add devicemodel (modelname)
"""
device_settings_header_fsl1_2 = ["accountid", "deviceuuid", "devicenationality", "uploadsequence", "firmwareversion", 
                          "systemtype", "readertype", "devicetype", "uploaddate"]

device_settings_header = ["accountid", "deviceuuid", "devicenationality", "uploadsequence", "firmwareversion", 
                          "systemtype", "readertype", "devicetype", "modelname", "uploaddate"]

# Join filtered device data with user data
device_settings_df = all_final_devices_df.join(user_df, all_final_devices_df['accountid'] == user_df['id'], "inner") \
    .withColumnRenamed("Country", "devicenationality").drop("ID").drop("DOB")

# Uploaddate & Uploadsequence
sequenced_device_settings_df = device_settings_df.withColumn("uploaddate", date_format(current_date(), date_string_format)) \
    .withColumn("uploadsequence", unix_timestamp().cast("string"))

# Cache since we joined with user, and will do future join with measurement
sequenced_device_settings_df.cache()

# Only write LDD/LL/L3 devices to device-settings.csv
device_settings_ldd_ll_df = sequenced_device_settings_df.filter(col("readertype").isin(['MobileApp', 'Reader']))

# Deidentify
device_settings_hashed_df = device_settings_ldd_ll_df.withColumn('accountid', deid('accountid')).withColumn('deviceuuid', deid('deviceuuid'))

#### Validate Attributes 

In [0]:
valid_device_settings_df = device_settings_hashed_df.filter(
    col("accountid").isNotNull() & (col("accountid") != "") & 
    col("deviceuuid").isNotNull() & (col("deviceuuid") != "") & 
    col("firmwareversion").isNotNull() & (col("firmwareversion") != "")
)

#### Split FSL1&2 and FSL3 + Write

In [0]:
# Split FSL1&2 / FSL3
device_settings_fsl1_df = valid_device_settings_df.filter(col("libretype") == "FSL1&2")
device_settings_fsl3_df = valid_device_settings_df.filter(col("libretype") == "FSL3")

# Pull needed columns
device_settings_hashed_final_fsl1_df = device_settings_fsl1_df.select(*device_settings_header_fsl1_2)
device_settings_hashed_final_fsl3_df = device_settings_fsl3_df.select(*device_settings_header)

# Write device-settings
write_measurement(device_settings_hashed_final_fsl1_df, 'device-settings', 'FSL1&2')
write_measurement(device_settings_hashed_final_fsl3_df, 'device-settings', 'FSL3')
print('Done writing Device-Settings')

## Join Measurement & Devices

#### Measurement-Devices Join

In [0]:
# Left Outer Join with device-settings
measurement_devices_df = measurement_df.alias('m').join(sequenced_device_settings_df.alias('d'), \
                            ((measurement_df['PatientID'] == sequenced_device_settings_df['accountid']) & (measurement_df['DeviceID'] == sequenced_device_settings_df['DeviceID'])), "left_outer")

# DeviceSettings accountid will be NULL for any records not satisfying join condition, these are unprocessed records
unprocessed_measurement_df = measurement_devices_df.filter(col("d.accountid").isNull())

# DeviceSettings accountid will not be NULL for any records satisfying join condition, these are processed records
processed_measurement_df = measurement_devices_df.filter(col("d.accountid").isNotNull())

# Get the total # of records, processed, unprocessed, and disparity
total = measurement_devices_df.count()
processed = processed_measurement_df.count()
unprocessed = unprocessed_measurement_df.count()
print(f"Total Records: {total}")
print(f"Unprocessed Records: {unprocessed}")
print(f"Processed Records: {processed}")
print(f"Difference: {total - processed - unprocessed}")

# Get the required columns from records to be processed, we create all Marshall CSVs from this file
filtered_measurements_df = processed_measurement_df.select("m.*", "d.deviceuuid", "d.firmwareversion", "d.devicetype", "d.libretype", "d.devicenationality", "d.uploadsequence", "d.uploaddate", "d.Settings_SerialNumber", "d.readertype")

# Rename PatientID, FactoryTimestamp, Timestamp
filtered_measurements_df = filtered_measurements_df.withColumn("accountid", col("PatientID")) \
    .withColumn("factoryrecorded", col("FactoryTimestamp")) \
    .withColumn("userrecorded", col("Timestamp"))

#### Unprocessed Measurement Records (Base)

In [0]:
# Write unprocessed measurements
# (this is initial unprocessed records, more will be appended after further validation for each Type of Measurement)
print("Writing unprocessed measurement records")
temp_measurement_header = ['PatientID', 'm.DeviceID', 'TimeStamp', 'Type', 'SubType', 'FactoryTimestamp', 'RecordNumber', 'W1', 'W2', 'Created', 'D', 'ND']
unprocessed_measurement_df = unprocessed_measurement_df.select(*temp_measurement_header)

write_unprocessed_records(unprocessed_measurement_df, 'measurement', 'overwrite')

## Create Scheduled CSV

#### Bit Masking

In [0]:
"""
Scheduled W1
# b'uuuu_uuuu_uedc_bbaa

e: canMerge
d: isFirstAfterActivation
c: isFirstAfterTimeChange
bb: lowOutOfRange
    0 -> null
    1 -> present, false
    2 -> invalid (not present, true)
    3 -> present, true
aa: highOutOfRange
    0 -> null
    1 -> present, false
    2 -> invalid (not present, true)
    3 -> present, true
"""
can_merge_mask           = 0b0100_0000
is_first_after_acv_mask  = 0b0010_0000
is_first_after_time_mask = 0b0001_0000
is_low_mask              = 0b0000_1100
is_high_mask             = 0b0000_0011

shift_low = 2

#### Process

In [0]:
print("Start Scheduled Processing")

headers = ["accountid", "factoryrecorded", "userrecorded", "deviceuuid", "devicenationality", "uploadsequence", 
            "firmwareversion", "mgdl", "firstaftertimechange", "devicetype", "sensoruid", "uploaddate", "sensoruidcorrected"]
"""
From measurement file: factoryrecorded, userrecorded, mgdl, firstaftertimechange, sensoruid, sensoruidcorrected
From device-settings: devicenationality, firmwareversion, devicetype
KEY: accountid, deviceuuid
"""

# Filter for only Type 0 (Scheduled) readings, and select the appropriate columns
scheduled_raw_df = filtered_measurements_df.filter(col("Type") == 0) # MIND THE DATATYPE

# Get mgdl from W2 by dividing by 10
scheduled_df = scheduled_raw_df.withColumn("mgdl", col("W2") / 10)

# Apply Masks over W1 & Bit Shift & Extract
scheduled_bit_extr_df = scheduled_df.withColumn("isLow", shiftRight(col("W1").bitwiseAND(is_low_mask), shift_low)) \
    .withColumn("isHigh", col("W1").bitwiseAND(is_high_mask)) \
    .withColumn("isFirstAfterTimeChange", when(col("W1").bitwiseAND(is_first_after_time_mask) > 0, True).otherwise(False)) \
    .withColumn("canMerge", when(col("W1").bitwiseAND(can_merge_mask) > 0, True).otherwise(False)) \
    .withColumn("isFirstAfterActivation", when(col("W1").bitwiseAND(is_first_after_acv_mask) > 0, True).otherwise(False)) \
    .withColumn("isLow", when(col("isLow") == 0, None).when(col("isLow") == 1, False).when(col("isLow") == 2, None).when(col("isLow") == 3, True)) \
    .withColumn("isHigh", when(col("isHigh") == 0, None).when(col("isHigh") == 1, False).when(col("isHigh") == 2, None).when(col("isHigh") == 3, True))

#### Get sensoruid & sensoruidcorrected

"""
LL canMerge doesn't have to be true.
But LDD canMerge must be true.
Drop any records where sensoruid is null.
"""

In [0]:
# Get sensoruid & sensoruidcorrected, when canMerge is true
scheduled_can_merge_df = scheduled_bit_extr_df.filter(col("canMerge") == True)

# Extract sensoruid & sensoruidcorrected
scheduled_sensors_df = scheduled_can_merge_df.withColumn("sensoruid", 
         when( (col("readertype") == "Reader") & (col("canMerge") == True), concat(get_sensoruid(col("RecordNumber")), col("accountid"))) \
        .when(col("readertype") == "MobileApp", concat(get_sensoruid(col("RecordNumber")), col("accountid"))) \
        .otherwise(None)) \
  .withColumn("sensoruidcorrected", 
         when( (col("readertype") == "Reader") & (col("canMerge") == True), concat(get_sensoruid_corrected(col("RecordNumber")), col("accountid"))) \
        .when(col("readertype") == "MobileApp", concat(get_sensoruid_corrected(col("RecordNumber")), col("accountid"))) \
        .otherwise(None))

#### Get lifecount

In [0]:
# Extract lifecount
scheduled_lifecount_df = scheduled_sensors_df.withColumn("lifecount", 
         when(col("sensoruidcorrected").isNotNull(), get_lifecount(col("RecordNumber"))) \
        .otherwise(None))

#### Validate Attributes

In [0]:
# Blank & Null check on [factoryrecorded, userrecorded, mgdl, sensoruid, firstaftertimechange]
# validate_iso on factoryrecorded & userrecorded

scheduled_valid_df = scheduled_lifecount_df.filter(
    col("factoryrecorded").isNotNull() & (col("factoryrecorded") != "") & (validate_iso(col("factoryrecorded")) == True) &
    col("userrecorded").isNotNull() & (col("userrecorded") != "") & (validate_iso(col("userrecorded")) == True)
)
    
scheduled_valid_df.cache()
print("Valid scheduled records: ", scheduled_valid_df.count())

# Get Invalid Records (all_records EXCEPT valid_records)
scheduled_invalid_df = scheduled_lifecount_df.exceptAll(scheduled_valid_df)

#### Deidentify

In [0]:
scheduled_deid_df = scheduled_valid_df.withColumn("accountid", deid(col("accountid"))) \
    .withColumn("deviceuuid", deid(col("deviceuuid"))) \
    .withColumn("sensoruid", deid(col("sensoruid"))) \
    .withColumn("sensoruidcorrected", deid(col("sensoruidcorrected")))

#### Split FSL1&2 / FSL3

In [0]:
# Split FSL1&2 / FSL3
scheduled_fsl1_df = scheduled_deid_df.filter(col("libretype") == "FSL1&2")
scheduled_fsl3_df = scheduled_deid_df.filter(col("libretype") == "FSL3")

#### Pull Final Columns

In [0]:
# headers = ["accountid", "factoryrecorded", "userrecorded", "deviceuuid", "devicenationality", "uploadsequence", "firmwareversion", "mgdl", "firstaftertimechange", "devicetype", "sensoruid", "uploaddate", "sensoruidcorrected"]

scheduled_final_fsl1_df = scheduled_fsl1_df.select(
    col("accountid"), 
    col("factoryrecorded"), 
    col("userrecorded"), 
    col("deviceuuid"), 
    col("devicenationality"), 
    col("uploadsequence"), 
    col("firmwareversion"),
    col("mgdl"), 
    col("isFirstAfterTimeChange").alias("firstaftertimechange"), 
    col("devicetype"), 
    col("sensoruid"), 
    col("uploaddate"), 
    col("sensoruidcorrected"))

scheduled_final_fsl3_df = scheduled_fsl3_df.select(
    col("accountid"), 
    col("factoryrecorded"), 
    col("userrecorded"), 
    col("deviceuuid"), 
    col("devicenationality"), 
    col("uploadsequence"), 
    col("firmwareversion"),
    col("mgdl"), 
    col("isFirstAfterTimeChange").alias("firstaftertimechange"), 
    col("devicetype"), 
    col("sensoruid"), 
    col("uploaddate"), 
    col("sensoruidcorrected"),
    col("lifecount"))

#### Write to S3

In [0]:
# Write Scheduled
write_measurement(scheduled_final_fsl1_df, 'scheduled', 'FSL1&2')
write_measurement(scheduled_final_fsl3_df, 'scheduled', 'FSL3')
print('Done Writing Scheduled')

#### Write Unprocessed (Measurement) Scheduled

In [0]:
# Append invalid scheduled measurements (append to unprocessed folder)
print("Appending invalid scheduled records to unprocessed folder")

# Write unprocessed scheduled records
scheduled_invalid_final_df = scheduled_invalid_df.select(*measurement_header)
write_unprocessed_records(scheduled_invalid_final_df, 'measurement', 'append')

## Create Unscheduled CSV

#### Bit Masking

In [0]:
"""
Unscheduled W1
# b'uuuu_gggg_fedc_bbaa

gggg: trendArrow (integer value of binary string, mapping below)
    0 -> Undetermined
    1 -> FallingQuickly
    2 -> Falling
    3 -> Stable
    4 -> Rising
    5 -> RisingQuickly
f: isStreaming
e: isViewed
d: isFirstAfterTimeChange
c: isActionable
bb: lowOutOfRange
    0 -> null
    1 -> present, false
    2 -> invalid (not present, true)
    3 -> present, true
aa: highOutOfRange
    0 -> null
    1 -> present, false
    2 -> invalid (not present, true)
    3 -> present, true
"""
trend_arrow_mask          = 0b1111_0000_0000
is_streaming_mask         = 0b0000_1000_0000
is_viewed_mask            = 0b0000_0100_0000
is_first_after_time_mask  = 0b0000_0010_0000
is_actionable_mask        = 0b0000_0001_0000
is_low_mask               = 0b0000_0000_1100
is_high_mask              = 0b0000_0000_0011

shift_trend = 8
shift_low = 2

#### Process

In [0]:
print("Start Current Processing")

"""
headers = [accountid, factoryrecorded, userrecorded, deviceuuid, devicenationality, uploadsequence, firmwareversion, mgdl, actionable, trendarrow, uploaddate]

From measurement file: factoryrecorded, userrecorded, mgdl, actionable, trendarrow
From device-settings: devicenationality, firmwareversion, devicetype
KEY: accountid, deviceuuid
"""

# Filter for only Type 0 (Unscheduled) readings, and select the appropriate columns
unscheduled_raw_df = filtered_measurements_df.filter(col("Type") == 1)

# Get mgdl from W2 by dividing by 10
unscheduled_df = unscheduled_raw_df.withColumn("mgdl", col("W2") / 10)

# Apply Masks over W1 & Bit Shift & Extract
unscheduled_bit_extr_df = unscheduled_df.withColumn("trendArrow", shiftRight(col("W1").bitwiseAND(trend_arrow_mask), shift_trend)) \
    .withColumn("isLow", shiftRight(col("W1").bitwiseAND(is_low_mask), shift_low)) \
    .withColumn("isHigh", col("W1").bitwiseAND(is_high_mask)) \
    .withColumn("isStreaming", when(col("W1").bitwiseAND(is_streaming_mask) > 0, True).otherwise(False)) \
    .withColumn("isViewed", when(col("W1").bitwiseAND(is_viewed_mask) > 0, True).otherwise(False)) \
    .withColumn("isFirstAfterTimeChange", when(col("W1").bitwiseAND(is_first_after_time_mask) > 0, True).otherwise(False)) \
    .withColumn("isActionable", when(col("W1").bitwiseAND(is_actionable_mask) > 0, True).otherwise(False)) \
    .withColumn("isLow", when(col("isLow") == 0, None).when(col("isLow") == 1, False).when(col("isLow") == 2, None).when(col("isLow") == 3, True)) \
    .withColumn("isHigh", when(col("isHigh") == 0, None).when(col("isHigh") == 1, False).when(col("isHigh") == 2, None).when(col("isHigh") == 3, True))

# Apply trendArrow mapping
unscheduled_trend_df = unscheduled_bit_extr_df.withColumn("trendArrow", 
     when(col("trendArrow") == 0, "Undetermined") \
    .when(col("trendArrow") == 1, "FallingQuickly") \
    .when(col("trendArrow") == 2, "Falling") \
    .when(col("trendArrow") == 3, "Stable") \
    .when(col("trendArrow") == 4, "Rising") \
    .when(col("trendArrow") == 5, "RisingQuickly") \
    .otherwise(None))

#### Validate Attributes

In [0]:
# NULL / BLANK check on [factoryrecorded, userrecorded, mgdl, actionable, trendarrow]
# validate_iso on [userrecorded, factoryrecorded]
unscheduled_valid_df = unscheduled_trend_df.filter(
    col("factoryrecorded").isNotNull() & (col("factoryrecorded") != "") & (validate_iso(col("factoryrecorded")) == True) &
    col("userrecorded").isNotNull() & (col("userrecorded") != "") & (validate_iso(col("userrecorded")) == True)
)

unscheduled_valid_df.cache()
print("Valid unscheduled records: ", unscheduled_valid_df.count())

# Get Invalid Records (all_records EXCEPT valid_records)
unscheduled_invalid_df = unscheduled_trend_df.exceptAll(unscheduled_valid_df)

#### Deidentify

In [0]:
unscheduled_deid_df = unscheduled_valid_df.withColumn("accountid", deid(col("accountid"))) \
    .withColumn("deviceuuid", deid(col("deviceuuid")))

#### Split FSL1&2 / FSL3

In [0]:
# Split FSL1&2 / FSL3
unscheduled_fsl1_df = unscheduled_deid_df.filter(col("libretype") == "FSL1&2")
unscheduled_fsl3_df = unscheduled_deid_df.filter(col("libretype") == "FSL3")

#### Pull Final Columns

In [0]:
# headers = [accountid, factoryrecorded, userrecorded, deviceuuid, devicenationality, 
#            uploadsequence, firmwareversion, mgdl, actionable, trendarrow, uploaddate]

# FSL3: Add highoutofrange, lowoutofrange, firstaftertimechange, viewed, streaming
unscheduled_final_fsl1_df = unscheduled_fsl1_df.select(
    col("accountid"), 
    col("factoryrecorded"), 
    col("userrecorded"), 
    col("deviceuuid"), 
    col("devicenationality"), 
    col("uploadsequence"), 
    col("firmwareversion"),
    col("mgdl"), 
    col("isActionable").alias("actionable"),
    col("trendArrow").alias("trendarrow"), 
    col("uploaddate"))

unscheduled_final_fsl3_df = unscheduled_fsl3_df.select(
    col("accountid"), 
    col("factoryrecorded"), 
    col("userrecorded"), 
    col("deviceuuid"), 
    col("devicenationality"), 
    col("uploadsequence"), 
    col("firmwareversion"),
    col("mgdl"), 
    col("isHigh").alias("highoutofrange"), 
    col("isLow").alias("lowoutofrange"), 
    col("isActionable").alias("actionable"),
    col("isFirstAfterTimeChange").alias("firstaftertimechange"),
    col("isViewed").alias("viewed"),
    col("isStreaming").alias("streaming"),
    col("trendArrow").alias("trendarrow"), 
    col("uploaddate"))

#### Write to S3

In [0]:
# Write Unscheduled
write_measurement(unscheduled_final_fsl1_df, 'current', 'FSL1&2')
write_measurement(unscheduled_final_fsl3_df, 'current', 'FSL3')
print('Done Writing Unscheduled')

#### Write Unprocessed (Measurement) Unscheduled

In [0]:
# Append invalid unscheduled measurements (append to unprocessed folder)
print("Appending invalid unscheduled records to unprocessed folder")

# Write insulin-device-data records
unscheduled_invalid_final_df = unscheduled_invalid_df.select(*measurement_header)
write_unprocessed_records(unscheduled_invalid_final_df, 'measurement', 'append')

## Create Food CSV

#### Process

In [0]:
"""
0buuuu_uuuu_uuuu_uaaa

aaa: ResultType (d5) truncated to 3 bits, values range [0,3]
"""

In [0]:
# Might be removing accountid

"""
headers = [accountid, factoryrecorded, userrecorded, deviceuuid, devicenationality,
           uploadsequence, firmwareversion, mealname, valuecarbs, numberofservings, uploaddate]

From measurement file: accountid, factoryrecorded, userrecorded, mealname, valuecarbs, numberofservings
From device-settings: deviceuuid, devicenationality, firmwareversion
KEY: accountid, deviceuuid
"""
print("Start Food Processing")

# Filter for only Type 0 (Food) readings, and select the appropriate columns
food_raw_df = filtered_measurements_df.filter(col("Type") == 5) # MIND THE DATATYPE

# Define the schema of the D JSON field
# d2: GramsCarbs
# d3: numberofservings
# d6: LinkedGlucoseRecordNumber
d_json_schema = "struct<d2: float, d3: int, d6: string>"

# Extract the gramsCarbs & numberofservings from D: d2
food_with_carbs_df = food_raw_df.withColumn("jsonData",from_json(col("D"), d_json_schema)) \
                .withColumn("valuecarbs", col("jsonData.d2")) \
                .withColumn("numberofservings", col("jsonData.d3")) \
                .withColumn("linkedglucoserecordnumber", col("jsonData.d6"))

# Get mealname from subtype
"""
FoodTypeUnknown   FoodType = 0
FoodTypeBreakfast FoodType = 1
FoodTypeLunch     FoodType = 2
FoodTypeDinner    FoodType = 3
FoodTypeSnack     FoodType = 4
"""
# MIND THE DATATYPES WHEN INGESTING PARQUET
food_with_meal_df = food_with_carbs_df.withColumn("mealname", 
     when(col('SubType') == 0, "Unknown") \
    .when(col('SubType') == 1, "Breakfast") \
    .when(col('SubType') == 2, "Lunch") \
    .when(col('SubType') == 3, "Dinner") \
    .when(col('SubType') == 4, "Snack"))                              

#### Validate Attributes

In [0]:
# NULL / BLANK check on [factoryrecorded, userrecorded, mealname]
# validate_iso on [userrecorded, factoryrecorded]
food_valid_df = food_with_meal_df.filter(
    col("factoryrecorded").isNotNull() & (col("factoryrecorded") != "") & (validate_iso(col("factoryrecorded")) == True) &
    col("userrecorded").isNotNull() & (col("userrecorded") != "") & (validate_iso(col("userrecorded")) == True)
)

food_valid_df.cache()
print("Valid food records: ", food_valid_df.count())

# Get Invalid Records (all_records EXCEPT valid_records)
food_invalid_df = food_with_meal_df.exceptAll(food_valid_df)

#### Deidentify

In [0]:
food_deid_df = food_valid_df.withColumn("accountid", deid(col("accountid"))) \
    .withColumn("deviceuuid", deid(col("deviceuuid")))

#### Split FSL1&2 / FSL3

In [0]:
# Split FSL1&2 / FSL3
food_fsl1_df = food_deid_df.filter(col("libretype") == "FSL1&2")
food_fsl3_df = food_deid_df.filter(col("libretype") == "FSL3")

#### Pull Final Columns

In [0]:
# headers = ['accountid', 'factoryrecorded', 'userrecorded', 'deviceuuid', 
#            'devicenationality', 'uploadsequence', 'firmwareversion', 'mealname', 
#            'valuecarbs', 'uploaddate']
# fsl3: Added numberofservings
food_final_fsl1_df = food_fsl1_df.select(
    col("accountid"), 
    col("factoryrecorded"), 
    col("userrecorded"), 
    col("deviceuuid"), 
    col("devicenationality"), 
    col("uploadsequence"), 
    col("firmwareversion"),
    col("mealname"), 
    col("valuecarbs"),
    col("uploaddate"))

food_final_fsl3_df = food_fsl3_df.select(
    col("accountid"), 
    col("factoryrecorded"), 
    col("userrecorded"), 
    col("deviceuuid"), 
    col("devicenationality"), 
    col("uploadsequence"), 
    col("firmwareversion"),
    col("mealname"), 
    col("valuecarbs"),
    col("numberofservings"),
    col("uploaddate"))

#### Write to S3

In [0]:
# Write Food
write_measurement(food_final_fsl1_df, 'food', 'FSL1&2')
write_measurement(food_final_fsl3_df, 'food', 'FSL3')
print('Done Writing Food')

#### Write Unprocessed (Measurement) Food

In [0]:
# Append invalid food measurements (append to unprocessed folder)
print("Appending invalid food records to unprocessed folder")

# Write insulin-device-data records
food_invalid_final_df = food_invalid_df.select(*measurement_header)
write_unprocessed_records(food_invalid_final_df, 'measurement', 'append')

## Create Generic CSV

#### Bit Masking

In [0]:
"""
Generic W1 (ISFGlucoseAlarm)
# b'uonm_lkji_hgfe_dcba

a: fixedlowglucosealarmisinepisode
b: fixedlowglucosealarmiscleared
c: fixedlowglucosealarmispresented
d: fixedlowglucosealarmisdismissed
e: fixedlowglucosealarmisusercleared

f: lowglucosealarmisinepisode
g: lowglucosealarmiscleared
h: lowglucosealarmispresented
i: lowglucosealarmisdismissed
j: lowglucosealarmisusercleared

k: highglucosealarmisinepisode
l: highglucosealarmiscleared
m: highglucosealarmispresented
n: highglucosealarmisdismissed
o: highglucosealarmisusercleared

Generic W1 (AlarmSetting)
a: islowglucoseenabled
b: ishighglucoseenabled
c: issignallossalarmenabled
d: isalarmsoundenabled
e: isalarmnotificationenabled
ffff_fff: configurationchangesource

Generic W2
# b'uuuu_uuuu_uufe_dbca

a: signallossalarmisinepisode
b: signallossalarmiscleared
c: signallossalarmispresented
d: signallossalarmisautodismissed
e: signallossalarmisuserdismissed
"""

# W1 (SubType 21): b'uonm_lkji_hgfe_dcba
fixed_low_episode_mask      = 0b0000_0000_0000_0001 # a
fixed_low_cleared_mask      = 0b0000_0000_0000_0010 # b
fixed_low_presented_mask    = 0b0000_0000_0000_0100 # c
fixed_low_dismissed_mask    = 0b0000_0000_0000_1000 # d
fixed_low_user_cleared_mask = 0b0000_0000_0001_0000 # e (not used)

low_episode_mask      = 0b0000_0000_0010_0000 # f
low_cleared_mask      = 0b0000_0000_0100_0000 # g
low_presented_mask    = 0b0000_0000_1000_0000 # h
low_dismissed_mask    = 0b0000_0001_0000_0000 # i
low_user_cleared_mask = 0b0000_0010_0000_0000 # j (not used)

high_episode_mask      = 0b0000_0100_0000_0000 # k
high_cleared_mask      = 0b0000_1000_0000_0000 # l
high_presented_mask    = 0b0001_0000_0000_0000 # m
high_dismissed_mask    = 0b0010_0000_0000_0000 # n
high_user_cleared_mask = 0b0010_0000_0000_0000 # o (not used)

# W1 (SubType 22): b'uuuf_ffff_fffe_dcba
low_glucose_enabled_mask   = 0b0000_0000_0000_0001 # a
high_glucose_enabled_mask  = 0b0000_0000_0000_0010 # b
signal_loss_enabled_mask   = 0b0000_0000_0000_0100 # c
alarm_sound_enabled_mask   = 0b0000_0000_0000_1000 # d
alarm_notif_enabled_mask   = 0b0000_0000_0001_0000 # e
config_data_source_mask    = 0b0000_1111_1110_0000 # ffff_fff

shift_config_src = 5

# W2 (SubType 21): b'uuuu_uuuu_uufe_dcba
signal_loss_episode_mask        = 0b0000_0000_0000_0001 # a
signal_loss_cleared_mask        = 0b0000_0000_0000_0010 # b
signal_loss_presented_mask      = 0b0000_0000_0000_0100 # c
signal_loss_auto_dismissed_mask = 0b0000_0000_0000_1000 # d
signal_loss_user_dismissed_mask = 0b0000_0000_0001_0000 # e
signal_loss_user_cleared_mask   = 0b0000_0000_0010_0000 # f

#### Process

In [0]:
"""
headers =  ['accountid', 'factoryrecorded', 'userrecorded', 'deviceuuid', 'devicenationality', 'uploadsequence', 'firmwareversion', 'type', 'localization', 
    'fixedlowglucosealarmisinepisode', 'fixedlowglucosealarmiscleared', 'fixedlowglucosealarmispresented', 'fixedlowglucosealarmisdismissed', 'fixedlowglucosealarmisusercleared',
    'lowglucosealarmisinepisode', 'lowglucosealarmiscleared', 'lowglucosealarmispresented', 'lowglucosealarmisdismissed', 'lowglucosealarmisusercleared',
    'highglucosealarmisinepisode', 'highglucosealarmiscleared', 'highglucosealarmispresented', 'highglucosealarmisdismissed', 'highglucosealarmisusercleared',
    'signallossalarmisinepisode', 'signallossalarmiscleared', 'signallossalarmispresented', 'signallossalarmisautodismissed', 'signallossalarmisuserdismissed', 'signallossalarmisusercleared',
    'lowglucosethresholdinmgdl', 'highglucosethresholdinmgdl',
    'islowglucoseenabled', 'ishighglucoseenabled', 'issignallossalarmenabled', 'isalarmsoundenabled',
    'isalarmnotificationenabled', 'configurationchangesource', 'value', 'timedelta', 'uploaddate', 'sensoruid', 'producttype',
    'puckgen', 'activationtype', 'wearduration', 'warmuptime', 'intensity', 'durationinminutes', 'sensorexpired', 'sensorterminated']

From measurement file: accountid, factoryrecorded, userrecorded, mealname, valuecarbs, ...
From device-settings: deviceuuid, devicenationality, firmwareversion
KEY: accountid, deviceuuid
"""
print("Start Generic Processing")

# Filter for only Type 6 (Generic) readings, and select the appropriate columns
generic_df = filtered_measurements_df.filter(col("Type") == 6) # MIND THE DATATYPE 

# Extract ISFGlucoseAlarm fields from W1 (when SubType is 21)
generic_with_glucose_alarm_df = generic_df \
    .withColumn("fixedlowglucosealarmisinepisode", 
                 when( (col("SubType") == 21) & (col("W1").bitwiseAND(fixed_low_episode_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W1").bitwiseAND(fixed_low_episode_mask) == 0), False) \
                .otherwise(None))   \
    .withColumn("fixedlowglucosealarmiscleared",  
                 when( (col("SubType") == 21) & (col("W1").bitwiseAND(fixed_low_cleared_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W1").bitwiseAND(fixed_low_cleared_mask) == 0), False) \
                .otherwise(None))   \
    .withColumn("fixedlowglucosealarmispresented", 
                 when( (col("SubType") == 21) & (col("W1").bitwiseAND(fixed_low_presented_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W1").bitwiseAND(fixed_low_presented_mask) == 0), False) \
                .otherwise(None)) \
    .withColumn("fixedlowglucosealarmisdismissed", 
                 when( (col("SubType") == 21) & (col("W1").bitwiseAND(fixed_low_dismissed_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W1").bitwiseAND(fixed_low_dismissed_mask) == 0), False) \
                .otherwise(None)) \
    .withColumn("fixedlowglucosealarmisusercleared", 
                 when( (col("SubType") == 21) & (col("W1").bitwiseAND(fixed_low_user_cleared_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W1").bitwiseAND(fixed_low_user_cleared_mask) == 0), False) \
                .otherwise(None)) \
    .withColumn("lowglucosealarmisinepisode", 
                 when( (col("SubType") == 21) & (col("W1").bitwiseAND(low_episode_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W1").bitwiseAND(low_episode_mask) == 0), False) \
                .otherwise(None)) \
    .withColumn("lowglucosealarmiscleared", 
                 when( (col("SubType") == 21) & (col("W1").bitwiseAND(low_cleared_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W1").bitwiseAND(low_cleared_mask) == 0), False) \
                .otherwise(None))   \
    .withColumn("lowglucosealarmispresented", 
                 when( (col("SubType") == 21) & (col("W1").bitwiseAND(low_presented_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W1").bitwiseAND(low_presented_mask) == 0), False) \
                .otherwise(None)) \
    .withColumn("lowglucosealarmisdismissed",
                 when( (col("SubType") == 21) & (col("W1").bitwiseAND(low_dismissed_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W1").bitwiseAND(low_dismissed_mask) == 0), False) \
                .otherwise(None)) \
    .withColumn("lowglucosealarmisusercleared",
                 when( (col("SubType") == 21) & (col("W1").bitwiseAND(low_user_cleared_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W1").bitwiseAND(low_user_cleared_mask) == 0), False) \
                .otherwise(None)) \
    .withColumn("highglucosealarmisinepisode", 
                when( (col("SubType") == 21) & (col("W1").bitwiseAND(high_episode_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W1").bitwiseAND(high_episode_mask) == 0), False) \
                .otherwise(None))   \
    .withColumn("highglucosealarmiscleared",
                 when( (col("SubType") == 21) & (col("W1").bitwiseAND(high_cleared_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W1").bitwiseAND(high_cleared_mask) == 0), False) \
                .otherwise(None))   \
    .withColumn("highglucosealarmispresented", 
                when( (col("SubType") == 21) & (col("W1").bitwiseAND(high_presented_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W1").bitwiseAND(high_presented_mask) == 0), False) \
                .otherwise(None)) \
    .withColumn("highglucosealarmisdismissed", 
                when( (col("SubType") == 21) & (col("W1").bitwiseAND(high_dismissed_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W1").bitwiseAND(high_dismissed_mask) == 0), False) \
                .otherwise(None)) \
    .withColumn("highglucosealarmisusercleared", 
                when( (col("SubType") == 21) & (col("W1").bitwiseAND(high_user_cleared_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W1").bitwiseAND(high_user_cleared_mask) == 0), False) \
                .otherwise(None))

# Extract signalLost fields from W2 (when SubType is 21)
generic_with_w2_df = generic_with_glucose_alarm_df \
    .withColumn("signallossalarmisinepisode", 
                 when( (col("SubType") == 21) & (col("W2").bitwiseAND(signal_loss_episode_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W2").bitwiseAND(signal_loss_episode_mask) == 0), False) \
                .otherwise(None)) \
    .withColumn("signallossalarmiscleared", 
                 when( (col("SubType") == 21) & (col("W2").bitwiseAND(signal_loss_cleared_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W2").bitwiseAND(signal_loss_cleared_mask) == 0), False) \
                .otherwise(None)) \
    .withColumn("signallossalarmispresented", 
                 when( (col("SubType") == 21) & (col("W2").bitwiseAND(signal_loss_presented_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W2").bitwiseAND(signal_loss_presented_mask) == 0), False) \
                .otherwise(None)) \
    .withColumn("signallossalarmisautodismissed", 
                 when( (col("SubType") == 21) & (col("W2").bitwiseAND(signal_loss_auto_dismissed_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W2").bitwiseAND(signal_loss_auto_dismissed_mask) == 0), False) \
                .otherwise(None)) \
    .withColumn("signallossalarmisuserdismissed", 
                 when( (col("SubType") == 21) & (col("W2").bitwiseAND(signal_loss_user_dismissed_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W2").bitwiseAND(signal_loss_user_dismissed_mask) == 0), False) \
                .otherwise(None)) \
    .withColumn("signallossalarmisusercleared", 
                 when( (col("SubType") == 21) & (col("W2").bitwiseAND(signal_loss_user_cleared_mask) > 0), True) \
                .when( (col("SubType") == 21) & (col("W2").bitwiseAND(signal_loss_user_cleared_mask) == 0), False) \
                .otherwise(None))


# Extract AlarmSetting fields from W1 (when SubType is 22)
config_data_source_mask    = 0b0000_1111_1110_0000 # ffff_fff
generic_alarm_setting_df = generic_with_w2_df \
    .withColumn("islowglucoseenabled", 
                 when( (col("SubType") == 22) & (col("W1").bitwiseAND(low_glucose_enabled_mask) > 0), True) \
                .when( (col("SubType") == 22) & (col("W1").bitwiseAND(low_glucose_enabled_mask) == 0), False) \
                .otherwise(None)) \
    .withColumn("ishighglucoseenabled", 
                 when( (col("SubType") == 22) & (col("W1").bitwiseAND(high_glucose_enabled_mask) > 0), True) \
                .when( (col("SubType") == 22) & (col("W1").bitwiseAND(high_glucose_enabled_mask) == 0), False) \
                .otherwise(None)) \
    .withColumn("issignallossalarmenabled", 
                 when( (col("SubType") == 22) & (col("W1").bitwiseAND(signal_loss_enabled_mask) > 0), True) \
                .when( (col("SubType") == 22) & (col("W1").bitwiseAND(signal_loss_enabled_mask) == 0), False) \
                .otherwise(None)) \
    .withColumn("isalarmsoundenabled", 
                 when( (col("SubType") == 22) & (col("W1").bitwiseAND(alarm_sound_enabled_mask) > 0), True) \
                .when( (col("SubType") == 22) & (col("W1").bitwiseAND(alarm_sound_enabled_mask) == 0), False) \
                .otherwise(None)) \
    .withColumn("isalarmnotificationenabled", 
                 when( (col("SubType") == 22) & (col("W1").bitwiseAND(alarm_notif_enabled_mask) > 0), True) \
                .when( (col("SubType") == 22) & (col("W1").bitwiseAND(alarm_notif_enabled_mask) == 0), False) \
                .otherwise(None)) \
    .withColumn("configurationchangesource", 
                 when((col("SubType") == 22), shiftRight(col("W1").bitwiseAND(config_data_source_mask), shift_config_src)) \
                .otherwise(None))

# Extract JSON column fields
# Define the schema of the D JSON field
# d28: Localization               -> localization
# d31: LowGlucoseThresholdInMgDl  -> lowglucosethresholdinmgdl
# d32: HighGlucoseThresholdInMgDl -> highglucosethresholdinmgdl
# d39: Value                      -> value
# d40: TimeDelta                  -> timedelta
# d45: ProductType                -> producttype
d_json_schema = """struct<d24: string, d25:string, d28: string, d29: string, d31: string, d32: string, d39: string, d40: string, 
                    d42: string, d43: string, d44: string, d45: string, d46: string, d47: string, d48: string, d49: string>"""

# Extract JSON fields from D
generic_json_fields_df = generic_alarm_setting_df.withColumn("jsonData",from_json(col("D"), d_json_schema)) \
        .withColumn("intensity", col("jsonData.d24")) \
        .withColumn("durationinminutes", col("jsonData.d25")) \
        .withColumn("localization", col("jsonData.d28")) \
        .withColumn("activationtype", col("jsonData.d29")) \
        .withColumn("lowglucosethresholdinmgdl", col("jsonData.d31")) \
        .withColumn("highglucosethresholdinmgdl", col("jsonData.d32")) \
        .withColumn("value", col("jsonData.d39")) \
        .withColumn("timedelta", col("jsonData.d40")) \
        .withColumn("puckgen", col("jsonData.d42")) \
        .withColumn("wearduration", col("jsonData.d43")) \
        .withColumn("latejoined", col("jsonData.d48")) \
        .withColumn("streaming", col("jsonData.d49")) \
        .withColumn("warmuptime", col("jsonData.d44")) \
        .withColumn("producttype", col("jsonData.d45")) \
        .withColumn("sensorexpired", col("jsonData.d46")) \
        .withColumn("sensorterminated", col("jsonData.d47")).drop("jsonData")

##### Get Generic Type

In [0]:
# supported_types = ['medication','alarmsetting','temperature','isfglucosealarm',
#                    'timechange','exercise','sensorstart','low','high','projectedlow','projectedhigh']

generic_with_type_df = generic_json_fields_df.withColumn("generictype", 
                                when(col("SubType") == 0, "exercise")    \
                               .when(col("SubType") == 1, "medication")  \
                               .when(col("SubType") == 2, "low")  \
                               .when(col("SubType") == 3, "high")  \
                               .when(col("SubType") == 4, "projectedlow")  \
                               .when(col("SubType") == 5, "projectedhigh")  \
                               .when(col("SubType") == 8, "sensorstart")  \
                               .when(col("SubType") == 10, "timechange") \
                               .when(col("SubType") == 21, "isfGlucoseAlarm") \
                               .when(col("SubType") == 22, "alarmSetting") \
                               .when(col("SubType") == 23, "temperature") \
                               .when(col("SubType") == 25, "sensorend") \
                               .otherwise(None))

##### Get sensoruid (correct method)

In [0]:
# Extract sensoruid from RecordNumber, use corrected method
generic_with_sensor_df = generic_with_type_df.withColumn("sensoruid", 
    when((col("readertype") == "MobileApp") & ((col("generictype") == "sensorstart") | (col("generictype") == "sensorend")), 
         concat(get_sensoruid_corrected(col("RecordNumber")), col("accountid"))).otherwise(None))

#### Validate Attributes

In [0]:
# NULL / BLANK check on [factoryrecorded, userrecorded, type]
# validate_iso on [userrecorded, factoryrecorded]
generic_valid_df = generic_with_sensor_df.filter(
    col("factoryrecorded").isNotNull() & (col("factoryrecorded") != "") & (validate_iso(col("factoryrecorded")) == True) &
    col("userrecorded").isNotNull() & (col("userrecorded") != "") & (validate_iso(col("userrecorded")) == True)
)

# Cache the filtering
generic_valid_df.cache()
generic_with_sensor_df.cache()

print("Valid generic records: ", generic_valid_df.count())

# Get Invalid Records (all_records EXCEPT valid_records)
generic_invalid_df = generic_with_sensor_df.exceptAll(generic_valid_df)

#### Deidentify

In [0]:
generic_deid_df = generic_valid_df.withColumn("accountid",  deid(col("accountid")))  \
                                  .withColumn("deviceuuid", deid(col("deviceuuid"))) \
                                  .withColumn("sensoruid",  deid(col("sensoruid")))

#### Split FSL1&2 / FSL3

In [0]:
generic_fsl1_df = generic_deid_df.filter(col("libretype") == "FSL1&2")
generic_fsl3_df = generic_deid_df.filter(col("libretype") == "FSL3")

#### Pull Final Columns

In [0]:
# FSL3: Added fixedlowglucosealarmisusercleared, lowglucosealarmisusercleared, highglucosealarmisusercleared, signallossalarmisusercleared,
# puckgen, activationtype, wearduration, warmuptime, intensity, durationInMinutes, sensorexpired, sensorterminated

fsl1_headers =  ['accountid', 'factoryrecorded', 'userrecorded', 'deviceuuid', 'devicenationality', 'uploadsequence', 'firmwareversion', col('generictype').alias('type'), 'localization', 
    'fixedlowglucosealarmisinepisode', 'fixedlowglucosealarmiscleared', 'fixedlowglucosealarmispresented', 'fixedlowglucosealarmisdismissed',
    'lowglucosealarmisinepisode', 'lowglucosealarmiscleared', 'lowglucosealarmispresented', 'lowglucosealarmisdismissed',
    'highglucosealarmisinepisode', 'highglucosealarmiscleared', 'highglucosealarmispresented', 'highglucosealarmisdismissed', 
    'signallossalarmisinepisode', 'signallossalarmiscleared', 'signallossalarmispresented', 'signallossalarmisautodismissed', 'signallossalarmisuserdismissed',
    'lowglucosethresholdinmgdl', 'highglucosethresholdinmgdl',
    'islowglucoseenabled', 'ishighglucoseenabled', 'issignallossalarmenabled', 'isalarmsoundenabled',
    'isalarmnotificationenabled', 'configurationchangesource', 'value', 'timedelta', 'uploaddate', 'sensoruid', 'producttype']

headers =  ['accountid', 'factoryrecorded', 'userrecorded', 'deviceuuid', 'devicenationality', 'uploadsequence', 'firmwareversion', 
            col('generictype').alias('type'), 'localization', 
    'fixedlowglucosealarmisinepisode', 'fixedlowglucosealarmiscleared', 'fixedlowglucosealarmispresented', 'fixedlowglucosealarmisdismissed', 'fixedlowglucosealarmisusercleared',
    'lowglucosealarmisinepisode', 'lowglucosealarmiscleared', 'lowglucosealarmispresented', 'lowglucosealarmisdismissed', 'lowglucosealarmisusercleared',
    'highglucosealarmisinepisode', 'highglucosealarmiscleared', 'highglucosealarmispresented', 'highglucosealarmisdismissed', 'highglucosealarmisusercleared',
    'signallossalarmisinepisode', 'signallossalarmiscleared', 'signallossalarmispresented', 'signallossalarmisautodismissed', 'signallossalarmisuserdismissed', 'signallossalarmisusercleared',
    'lowglucosethresholdinmgdl', 'highglucosethresholdinmgdl',
    'islowglucoseenabled', 'ishighglucoseenabled', 'issignallossalarmenabled', 'isalarmsoundenabled',
    'isalarmnotificationenabled', 'configurationchangesource', 'value', 'timedelta', 'uploaddate', 'sensoruid', 'producttype',
    'puckgen', 'activationtype', 'wearduration', 'latejoined', 'streaming', 'warmuptime', 'intensity', 'durationinminutes', 'sensorexpired', 'sensorterminated']

generic_final_fsl1_df = generic_fsl1_df.select(*fsl1_headers)
generic_final_fsl3_df = generic_fsl3_df.select(*headers)

#### Write to S3

In [0]:
# Write Generic
write_measurement(generic_final_fsl1_df, 'generic', 'FSL1&2')
write_measurement(generic_final_fsl3_df, 'generic', 'FSL3')
print('Done Writing Generic')

#### Write Unprocessed (Measurement) Generic

In [0]:
# Append invalid generic measurements (append to unprocessed folder)
print("Appending invalid generic records to unprocessed folder")

# Write insulin-device-data records
generic_invalid_final_df = generic_invalid_df.select(*measurement_header)
write_unprocessed_records(generic_invalid_final_df, 'measurement', 'append')

## Create Insulin CSV

#### Process

In [0]:
"""
headers = ['accountid', 'factoryrecorded', 'userrecorded', 'deviceuuid', 'devicenationality', 'uploadsequence', 'firmwareversion', 'type'
    'units','insulinonboardinunits', 'correctionamountinunits', 'mealamountinunits', 'useroverrideinunits', 'uploaddate']

From measurement file: accountid, factoryrecorded, userrecorded, type, units, insulinonboardinunits, correctionamountinunits, mealamountinunits, useroverrideinunits
From device-settings: deviceuuid, devicenationality, firmwareversion
KEY: accountid, deviceuuid
"""

In [0]:
print("Start Insulin Processing")

# Filter for only Type 4 (Insulin) readings & not ConnectedDevice InsulinLog reading, and select the appropriate columns
insulin_raw_df = filtered_measurements_df.filter( (col("Type") == 4) & (col("devicetype") != "ConnectedPen")) # MIND THE DATATYPE 

# Get the insulin type
"""
    InsulinTypeRapidActing = 0
    InsulinTypeLongActing  = 1
    InsulinTypeOther       = 2
"""
insulin_with_type_df = insulin_raw_df.withColumn("insulintype", 
                                                  when(col("SubType") == 0, "RapidActing") \
                                                 .when(col("SubType") == 1, "LongActing") \
                                                 .when(col("SubType") == 2, "Other"))

# Get units field by dividing W2 by 10
insulin_with_units_df = insulin_with_type_df.withColumn("units", col("W2") / 10)

# Extract JSON column fields
# Define the schema of the D JSON field
# d3: UserOverrideAmountInUnits  -> useroverrideinunits
# d4: MealAmountInUnits          -> mealamountinunits
# d5: CorrectionAmountInUnits    -> correctionamountsinunits
# d6: InsulinOnBoardInUnits      -> insulinonboardinunits
d_json_schema = """struct<d3: string, d4: string, d5: string, d6: string, d10: string>"""

# Extract JSON fields from D
insulin_json_fields_df = insulin_with_units_df.withColumn("jsonData",from_json(col("D"), d_json_schema)) \
        .withColumn("useroverrideinunits", col("jsonData.d3")) \
        .withColumn("mealamountinunits", col("jsonData.d4")) \
        .withColumn("correctionamountinunits", col("jsonData.d5")) \
        .withColumn("insulinonboardinunits", col("jsonData.d6")) \
        .withColumn("linkedglucoserecordnumber", col("jsonData.d10"))

#### Validate Attributes

In [0]:
# NULL / BLANK check on [factoryrecorded, userrecorded, units, type]
# validate_iso on [userrecorded, factoryrecorded]
insulin_valid_df = insulin_json_fields_df.filter(
    col("factoryrecorded").isNotNull() & (col("factoryrecorded") != "") & (validate_iso(col("factoryrecorded")) == True) &
    col("userrecorded").isNotNull() & (col("userrecorded") != "") & (validate_iso(col("userrecorded")) == True) # &
    # col("units").isNotNull() & 
    # col("insulintype").isNotNull() & (col("insulintype") != "") # '& 
    # col("sensoruid").isNotNull() & (col("sensoruid") != "")
)

insulin_valid_df.cache()
print("Valid insulin records: ", insulin_valid_df.count())

# Get Invalid Records (all_records EXCEPT valid_records)
insulin_invalid_df = insulin_json_fields_df.exceptAll(insulin_valid_df)

#### Deidentify

In [0]:
insulin_deid_df = insulin_valid_df.withColumn("accountid",  deid(col("accountid")))  \
                                .withColumn("deviceuuid", deid(col("deviceuuid")))

#### Split FSL1&2 / FSL3

In [0]:
insulin_fsl1_df = insulin_deid_df.filter(col("libretype") == "FSL1&2")
insulin_fsl3_df = insulin_deid_df.filter(col("libretype") == "FSL3")

#### Pull Final Columns

In [0]:
# headers = ['accountid', 'factoryrecorded', 'userrecorded', 'deviceuuid', 'devicenationality', 'uploadsequence', 'firmwareversion', 'type',
#     'units','insulinonboardinunits', 'correctionamountsinunits', 'mealamountinunits', 'useroverrideinunits', 'sensoruid', 'uploaddate']

insulin_final_fsl1_df = insulin_fsl1_df.select(
    col("accountid"),
    col("factoryrecorded"),
    col("userrecorded"),
    col("deviceuuid"),
    col("devicenationality"),
    col("uploadsequence"),
    col("firmwareversion"),
    col("insulintype").alias('type'),
    col("units"),
    col("insulinonboardinunits"),
    col("correctionamountinunits"),
    col("mealamountinunits"),
    col("useroverrideinunits"),
    col("uploaddate")
)

insulin_final_fsl3_df = insulin_fsl3_df.select(
    col("accountid"),
    col("factoryrecorded"),
    col("userrecorded"),
    col("deviceuuid"),
    col("devicenationality"),
    col("uploadsequence"),
    col("firmwareversion"),
    col("insulintype").alias('type'),
    col("units"),
    col("insulinonboardinunits"),
    col("correctionamountinunits"),
    col("mealamountinunits"),
    col("useroverrideinunits"),
    col("uploaddate")
)

#### Write to S3

In [0]:
# Write Insulin
write_measurement(insulin_final_fsl1_df, 'insulin', 'FSL1&2')
write_measurement(insulin_final_fsl3_df, 'insulin', 'FSL3')
print('Done Writing Insulin')

#### Write Unprocessed (Measurement) Insulin

In [0]:
# Append invalid insulin measurements (append to unprocessed folder)
print("Appending invalid insulin records to unprocessed folder")

# Write insulin-device-data records
insulin_invalid_final_df = insulin_invalid_df.select(*measurement_header)
write_unprocessed_records(insulin_invalid_final_df, 'measurement', 'append')

## Create Ketone CSV

#### Process

In [0]:
"""
headers = ['accountid', 'factoryrecorded', 'userrecorded', 'deviceuuid', 'devicenationality', 
           'uploadsequence', 'firmwareversion', 'valuemmol', 'uploaddate']

From measurement file: accountid, factoryrecorded, userrecorded, valuemmol
From device-settings: deviceuuid, devicenationality, firmwareversion
KEY: accountid, deviceuuid
"""

In [0]:
print("Start Ketone Processing")

# Filter for only Type 3 (Ketone) readings, and select the appropriate columns
ketone_raw_df = filtered_measurements_df.filter(col("Type") == 3) # MIND THE DATATYPE 

# Get valuemmol by dividing W1 by 10
ketone_df = ketone_raw_df.withColumn("valuemmol", col("W1") / 10)

#### Validate Attributes

In [0]:
# NULL / BLANK check on [factoryrecorded, userrecorded, valuemmol]
# validate_iso on [userrecorded, factoryrecorded]
ketone_valid_df = ketone_df.filter(
    col("factoryrecorded").isNotNull() & (col("factoryrecorded") != "") & (validate_iso(col("factoryrecorded")) == True) &
    col("userrecorded").isNotNull() & (col("userrecorded") != "") & (validate_iso(col("userrecorded")) == True)
)

ketone_valid_df.cache()
print("Valid ketone records: ", ketone_valid_df.count())

# Get Invalid Records (all_records EXCEPT valid_records)
ketone_invalid_df = ketone_df.exceptAll(ketone_valid_df)

#### Deidentify

In [0]:
ketone_deid_df = ketone_valid_df.withColumn("accountid",  deid(col("accountid")))  \
                                  .withColumn("deviceuuid", deid(col("deviceuuid")))

#### Split FSL1&2 / FSL3

In [0]:
ketone_fsl1_df = ketone_deid_df.filter(col("libretype") == "FSL1&2")
ketone_fsl3_df = ketone_deid_df.filter(col("libretype") == "FSL3")

#### Pull Final Columns

In [0]:
headers = ['accountid', 'factoryrecorded', 'userrecorded', 'deviceuuid', 'devicenationality', 
           'uploadsequence', 'firmwareversion', 'valuemmol', 'uploaddate']

ketone_final_fsl1_df = ketone_fsl1_df.select(*headers)
ketone_final_fsl3_df = ketone_fsl3_df.select(*headers)

#### Write to S3

In [0]:
# Write Ketone
write_measurement(ketone_final_fsl1_df, 'ketone', 'FSL1&2')
write_measurement(ketone_final_fsl3_df, 'ketone', 'FSL3')
print('Done Writing Ketone')

#### Write Unprocessed (Measurement) Ketone

In [0]:
# Append invalid ketone measurements (append to unprocessed folder)
print("Appending invalid ketone records to unprocessed folder")

# Write insulin-device-data records
ketone_invalid_final_df = ketone_invalid_df.select(*measurement_header)
write_unprocessed_records(ketone_invalid_final_df, 'measurement', 'append')

## Create Strip CSV

#### Bit Masking

In [0]:
"""
Strip W1
# b'uuuu_uuuu_uuuc_bbaa

c: IsFastingTest (d2)
bb: Nillable bool (2 bits) for CappedMeasurement.IsLow (lo)
aa: Nillable bool (2 bits) for CappedMeasurement.IsHigh (hi)
"""

# W1: b'uonm_lkji_hgfe_dcba
is_high_mask    = 0b0000_0000_0000_0011 # aa
is_low_mask     = 0b0000_0000_0000_1100 # bb
is_fasting_mask = 0b0000_0000_0001_0000 # c

shift_low = 2

#### Process

In [0]:
"""
headers = ['accountid', 'factoryrecorded', 'userrecorded', 'deviceuuid', 'devicenationality'
'uploadsequence', 'firmwareversion', 'mgdl', 'uploaddate']

From measurement file: accountid, factoryrecorded, userrecorded, mgdl
From device-settings: deviceuuid, devicenationality, firmwareversion
KEY: accountid, deviceuuid
"""

In [0]:
print("Start Strip Processing")

# Filter for only Type 2 (BloodGlucose) readings, and select the appropriate columns
strip_raw_df = filtered_measurements_df.filter(col("Type") == 2) # MIND THE DATATYPE 

# Get mgdl from W2 by dividing by 10
strip_df = strip_raw_df.withColumn("mgdl", col("W2") / 10)

# Apply Masks over W1 & Bit Shift & Extract
strip_bit_extr_df = strip_df.withColumn("isHigh", col("W1").bitwiseAND(is_high_mask)) \
    .withColumn("isLow", shiftRight(col("W1").bitwiseAND(is_low_mask), shift_low)) \
    .withColumn("isFastingTest", when(col("W1").bitwiseAND(is_fasting_mask) > 0, True).otherwise(False)) \
    .withColumn("isLow", when(col("isLow") == 0, None).when(col("isLow") == 1, False).when(col("isLow") == 2, None).when(col("isLow") == 3, True)) \
    .withColumn("isHigh", when(col("isHigh") == 0, None).when(col("isHigh") == 1, False).when(col("isHigh") == 2, None).when(col("isHigh") == 3, True))

In [0]:
# Get mealTime
"""
    MealTimeTypeNone   MealTimeType = 0
    MealTimeTypeBefore MealTimeType = 1
    MealTimeTypeAfter  MealTimeType = 2
    MealTimeTypeOther  MealTimeType = 3
"""
strip_meal_df = strip_bit_extr_df.withColumn("mealTime", 
                                     when(col("SubType") == 0, "None")   \
                                    .when(col("SubType") == 1, "Before") \
                                    .when(col("SubType") == 2, "After")  \
                                    .when(col("SubType") == 3, "Other"))

#### Validate Attributes

In [0]:
# NULL / BLANK check on [factoryrecorded, userrecorded, mgdl]
# validate_iso on [userrecorded, factoryrecorded]
strip_valid_df = strip_df.filter(
    col("factoryrecorded").isNotNull() & (col("factoryrecorded") != "") & (validate_iso(col("factoryrecorded")) == True) &
    col("userrecorded").isNotNull() & (col("userrecorded") != "") & (validate_iso(col("userrecorded")) == True) # &
    # col("mgdl").isNotNull()
)

strip_valid_df.cache()
print("Valid strip records: ", strip_valid_df.count())

# Get Invalid Records (all_records EXCEPT valid_records)
strip_invalid_df = strip_df.exceptAll(strip_valid_df)

#### Deidentify

In [0]:
strip_deid_df = strip_valid_df.withColumn("accountid",  deid(col("accountid")))  \
                              .withColumn("deviceuuid", deid(col("deviceuuid")))

#### Split FSL1&2 / FSL3

In [0]:
strip_fsl1_df = strip_deid_df.filter(col("libretype") == "FSL1&2")
strip_fsl3_df = strip_deid_df.filter(col("libretype") == "FSL3")

#### Pull Final Columns

In [0]:
headers = ['accountid', 'factoryrecorded', 'userrecorded', 'deviceuuid', 'devicenationality',
           'uploadsequence', 'firmwareversion', 'mgdl', 'uploaddate']
strip_final_fsl1_df = strip_fsl1_df.select(*headers)
strip_final_fsl3_df = strip_fsl3_df.select(*headers)

#### Write to S3

In [0]:
# Write Strip
write_measurement(strip_final_fsl1_df, 'strip', 'FSL1&2')
write_measurement(strip_final_fsl3_df, 'strip', 'FSL3')
print('Done Writing Strip')

#### Write Unprocessed (Measurement) Strip

In [0]:
# Append invalid strip measurements (append to unprocessed folder)
print("Appending invalid strip records to unprocessed folder")

# Write insulin-device-data records
strip_invalid_final_df = strip_invalid_df.select(*measurement_header)
write_unprocessed_records(strip_invalid_final_df, 'measurement', 'append')

## Insulin-Device

#### Get Insulin-Device data

In [0]:
print("Start Insulin-Device Processing")
insulin_device_df = sequenced_device_settings_df.filter(col("devicetype") == "ConnectedPen")

#### Deidentify

In [0]:
# Deidentified serialnumber is created from deid(serialnumber + accountid)
insulin_device_deid_df = insulin_device_df.withColumn("serialnumber", deid(concat(col("Settings_SerialNumber"), col("accountid")))) \
    .withColumn("accountid", deid(col("accountid")))  \
    .withColumn("deviceuuid", deid(col("deviceuuid")))

#### Split FSL1&2 / FSL3

In [0]:
# We write entire dataframe to both partitions
insulin_device_fsl1_df = insulin_device_deid_df
insulin_device_fsl3_df = insulin_device_deid_df

#### Select Columns

In [0]:
# Select only the columns we need & rename as needed
insulin_device_final_fsl1_df = insulin_device_fsl1_df.select(
    col("accountid"),
    col("deviceuuid"),
    col("devicenationality"),
    col("Manufacturer").alias("manufacturer"),
    col("Settings_ModelName").alias("modelname"),
    col("LocalModelName").alias("localmodelname"),
    col("serialnumber"),
    col("HardwareRevision").alias("hardwarerevision"),
    col("FirmwareRevision").alias("firmwarerevision"),
    col("SoftwareRevision").alias("softwarerevision"),
    col("BLESoftwareRevision").alias("blesoftwarerevision"),
    col("BLEProtocolRevision").alias("bleprotocolrevision"),
    col("DeviceTime").alias("devicetime"),
    col("DefaultInsulinType").alias("defaultinsulintype"),
    col("DefaultInsulinBrand").alias("defaultinsulinbrand"),
    col("uploadsequence"),
    col("uploaddate")
)

insulin_device_final_fsl3_df = insulin_device_fsl3_df.select(
    col("accountid"),
    col("deviceuuid"),
    col("devicenationality"),
    col("Manufacturer").alias("manufacturer"),
    col("Settings_ModelName").alias("modelname"),
    col("LocalModelName").alias("localmodelname"),
    col("serialnumber"),
    col("HardwareRevision").alias("hardwarerevision"),
    col("FirmwareRevision").alias("firmwarerevision"),
    col("SoftwareRevision").alias("softwarerevision"),
    col("BLESoftwareRevision").alias("blesoftwarerevision"),
    col("BLEProtocolRevision").alias("bleprotocolrevision"),
    col("DeviceTime").alias("devicetime"),
    col("DefaultInsulinType").alias("defaultinsulintype"),
    col("DefaultInsulinBrand").alias("defaultinsulinbrand"),
    col("uploadsequence"),
    col("uploaddate")
)

#### Write to S3

In [0]:
# Write Insulin-Device
write_measurement(insulin_device_final_fsl1_df, 'insulin-device', 'FSL1&2')
write_measurement(insulin_device_final_fsl3_df, 'insulin-device', 'FSL3')
print('Done Writing Insulin-Device')

## Insulin-Device-Data


#### Bit Masking

In [0]:
"""
Insulin-Device-Data W1
# b'uuuu_uuuu_fedc_baaa

f: Connected
e: PrimeAlgo
d: Invalid
c: EditedDose
b: PrimeDose
aaa: InsulinRecordType truncated to 3 bits, values range [0,2]
    0 -> InsulinRecordTypeQuickTag 
    1 -> InsulinRecordTypeManualInsulin
    2 -> InsulinRecordTypeCalculatorInsulin
"""

connected_mask    = 0b1000_0000
prime_algo_mask   = 0b0100_0000
invalid_mask      = 0b0010_0000
edited_dose_mask  = 0b0001_0000
prime_dose_mask   = 0b0000_1000
record_type_mask  = 0b0000_0111

#### Process

In [0]:
print("Start Insulin-Device-Data Processing")

# Filter for only ConnectedPen readings, and select the appropriate columns
insulin_device_data_raw_df = filtered_measurements_df.filter(col("devicetype") == "ConnectedPen")

# Apply Masks over W1 & Extract
insulin_device_data_bit_extr_df = insulin_device_data_raw_df.withColumn("IRT", col("W1").bitwiseAND(record_type_mask)) \
    .withColumn("Connected", when(col("W1").bitwiseAND(connected_mask) > 0, True).otherwise(False)) \
    .withColumn("PrimeAlgo", when(col("W1").bitwiseAND(prime_algo_mask) > 0, True).otherwise(False)) \
    .withColumn("Invalid", when(col("W1").bitwiseAND(invalid_mask) > 0, True).otherwise(False)) \
    .withColumn("EditedDose", when(col("W1").bitwiseAND(edited_dose_mask) > 0, True).otherwise(False)) \
    .withColumn("PrimeDose", when(col("W1").bitwiseAND(prime_dose_mask) > 0, True).otherwise(False)) \
    .withColumn("InsulinRecordType", 
                when(col("IRT") == 0, "InsulinRecordTypeQuickTag") \
                .when(col("IRT") == 1, "InsulinRecordTypeManualInsulin") \
                .when(col("IRT") == 2, "InsulinRecordTypeCalculatorInsulin") \
                .otherwise(None))

insulin_device_data_type_df = insulin_device_data_bit_extr_df.withColumn("insulintype", 
                                                  when(col("SubType") == 0, "RapidActing") \
                                                 .when(col("SubType") == 1, "LongActing") \
                                                 .when(col("SubType") == 2, "Other"))

# Extract JSON column fields
# Define the schema of the D JSON field
# d11: value              -> value
# d15: statusFlags        -> statusflags
# d16: elapsedSeconds     -> elapsedseconds
# d18: relativeTimestamp  -> relativetimestamp

d_json_schema = """struct<d11: string, d15: string, d16: string, d18: string>"""
# Extract JSON fields from D
insulin_device_data_d_fields_df = insulin_device_data_type_df.withColumn("jsonData",from_json(col("D"), d_json_schema)) \
        .withColumn("value", col("jsonData.d11")) \
        .withColumn("statusflags", col("jsonData.d15")) \
        .withColumn("elapsedseconds", col("jsonData.d16")) \
        .withColumn("relativetimestamp", col("jsonData.d18"))

# Define the schema of the ND JSON field                  
# d17:  insulinBrand       -> insulinbrand
nd_json_schema = """struct<d17: string>"""
# Extract JSON fields from ND
insulin_device_data_nd_fields_df = insulin_device_data_d_fields_df.withColumn("jsonData",from_json(col("ND"), nd_json_schema)) \
        .withColumn("insulinbrand", col("jsonData.d17"))


#### Validate Attributes

In [0]:
# ParseError if insulin_log_data.invalid is False
# validate_iso on [localtimestamp, utctimestamp]
insulin_device_data_valid_df = insulin_device_data_nd_fields_df.filter(
    col("FactoryTimestamp").isNotNull() & (col("FactoryTimestamp") != "") & (validate_iso(col("FactoryTimestamp")) == True) &
    col("Timestamp").isNotNull() & (col("Timestamp") != "") & (validate_iso(col("Timestamp")) == True) &
    (col("Invalid") == False)
)

insulin_device_data_valid_df.cache()
print("Valid insulin-device-data records: ", insulin_device_data_valid_df.count())

# Get Invalid Records
insulin_device_data_invalid_df = insulin_device_data_nd_fields_df.exceptAll(insulin_device_data_valid_df)

#### De-identify

In [0]:
# Deidentified serialnumber is created from deid(serialnumber + accountid)
insulin_device_data_deid_df = insulin_device_data_valid_df.withColumn("serialnumber", deid(concat(col("Settings_SerialNumber"), col("accountid")))) \
    .withColumn("accountid", deid(col("accountid"))) \
    .withColumn("deviceuuid", deid(col("deviceuuid")))

#### Split FSL1&2 / FSL3

In [0]:
insulin_device_data_fsl1_df = insulin_device_data_deid_df
insulin_device_data_fsl3_df = insulin_device_data_deid_df

#### Select Final Columns

In [0]:
# header = ['accountid', 'deviceuuid', 'devicenationality', 'relativetimestamp', 'elapsedseconds', 'serialnumber', 
#           'localtimestamp', 'utctimestamp', 'value', 'insulintype', 'insulinbrand', 'primedose', 'primealgo', 
#           'editdose', 'statusflags', 'uploadsequence', 'uploaddate']

insulin_device_data_final_fsl1_df = insulin_device_data_fsl1_df.select(
    col("accountid"),
    col("deviceuuid"),
    col("devicenationality"),
    col("relativetimestamp"),
    col("elapsedseconds"),
    col("serialnumber"),
    col("userrecorded").alias("localtimestamp"),
    col("factoryrecorded").alias("utctimestamp"),
    col("value"),
    col("insulintype"),
    col("insulinbrand"),
    col("primedose"),
    col("primealgo"),
    col("editeddose"),
    col("statusflags"),
    col("uploadsequence"),
    col("uploaddate")
)

insulin_device_data_final_fsl3_df = insulin_device_data_fsl3_df.select(
    col("accountid"),
    col("deviceuuid"),
    col("devicenationality"),
    col("relativetimestamp"),
    col("elapsedseconds"),
    col("serialnumber"),
    col("userrecorded").alias("localtimestamp"),
    col("factoryrecorded").alias("utctimestamp"),
    col("value"),
    col("insulintype"),
    col("insulinbrand"),
    col("primedose"),
    col("primealgo"),
    col("editeddose"),
    col("statusflags"),
    col("uploadsequence"),
    col("uploaddate")
)

#### Write to S3

In [0]:
# Write Insulin-Device-Data
write_measurement(insulin_device_data_final_fsl1_df, 'insulin-device-data', 'FSL1&2')
write_measurement(insulin_device_data_final_fsl3_df, 'insulin-device-data', 'FSL3')
print('Done Writing Insulin-Device-Data')

#### Write Unprocessed (Measurement) Insulin-Device-Data

In [0]:
# Append invalid insulin-device-data measurements (append to unprocessed folder)
print("Appending invalid insulin-device-data records to unprocessed folder")

# Write insulin-device-data records
insulin_device_data_invalid_final_df = insulin_device_data_invalid_df.select(*measurement_header)
write_unprocessed_records(insulin_device_data_invalid_final_df, 'measurement', 'append')

## Update Batch Status

In [0]:
from boto3.dynamodb.conditions import Key
from botocore.exceptions import ClientError

In [0]:
print(f"Start Batch Update for : {BATCH}")

# Update Process-Status Table, Set Batch as Processed
status_handler = SuperiorProcessStatusHandler(AWS_REGION, STATUS_TABLE, SNS_TOPIC, LOAD_DATE)
status_handler.update_batch_status(BATCH, 'Processed')
print(f"Processed Batch : {BATCH}")

In [0]:
dbutils.notebook.exit(BATCH)